# Advent of Code 2020

<https://adventofcode.com/2020>


## Setup

In [1]:
# imports
import re

from collections.abc import Iterator
from dataclasses import dataclass
from pathlib import Path
from itertools import combinations
from typing import Literal, Optional

In [2]:
# useful globals
DATA_DIR=Path("aoc_202/data")

In [3]:
# helper functions

def lines(day: int) -> Iterator[str]:
    input: Path = DATA_DIR / f"{day}.txt"
    with input.open() as f:
        for line in f:
            yield line.strip()

## Day 1

Given a list of numbers, find the two that _sum_ to 2020, then multiply them to get the final answer.

I guess I can just brute-force this, and use a generator over the lines 

In [4]:
numbers = []
for line in lines(1):
    number = int(line)
    for n in numbers:
        if n + number == 2020:
            print(n * number)
            break
    numbers.append(number)

FileNotFoundError: [Errno 2] No such file or directory: 'data/1.txt'

In [5]:
seen = []
for line in lines(1):
    current = int(line)
    for a, b in combinations(seen, 2):
        if (a + b + current) == 2020:
            print(a*b*current)
            break
    seen.append(current)

49214880


## Day 2

A password policy "1-3 a" means the letter "a" must appear 1-3 times in the password. How many policy-password pairs from the file are valid?

In [6]:
@dataclass
class Policy:
    letter: str
    min: int
    max: int

    def is_valid(self, password: str) -> bool:
        n = password.count(self.letter)
        return (self.min <= n) and (n <= self.max)

    def is_valid2(self, password: str) -> bool:
        return (password[self.min-1] == self.letter) ^ (password[self.max-1] == self.letter)

    @classmethod
    def from_line(cls, line: str):
        """1-3 a"""
        numbers, letter = line.split(" ")
        min, max = [int(x) for x in numbers.split("-")]
        return cls(letter, min, max)

In [7]:
valid = [0, 0]
for line in lines(2):
    policy_text, password = line.split(": ")
    policy = Policy.from_line(policy_text)
    if policy.is_valid(password):
        valid[0] += 1
    if policy.is_valid2(password):
        valid[1] += 1

print(valid)

[460, 251]


## Day 3

In [8]:
column = 0
trees = 0
for line in lines(3):
    row = [x == "#" for x in line]
    trees += row[column % len(row)]
    column += 3

print(trees)


265


### Part 2

Welp now we have to totally restructure part 1 so it's easier to input different row/column strides.

In [9]:
landscape: list[list[bool]] = []
for line in lines(3):
    row = [x == "#" for x in line]
    landscape.append(row)

def trees_in_path(landscape, right: int, down: int) -> int:
    trees = 0
    row, column = 0, 0
    while row < len(landscape):
        trees += landscape[row][column % len(landscape[0])]
        row += down
        column += right
    return trees

paths = [
    (1, 1),
    (3, 1),
    (5, 1),
    (7, 1),
    (1, 2)
]

total=1
for path in paths:
    total *= trees_in_path(landscape, *path)

print(total)

3154761400


In [10]:
len(landscape), len(landscape[0])

(323, 31)

# Day 4

Looks like we have to get a bit weirder with our parsing...

In [11]:
required_fields = (
    "byr",
    "iyr",
    "eyr",
    "hgt",
    "hcl",
    "ecl",
    "pid",
    # "cid",  # optional
)

valid = 0
passport = ""
for line in lines(4):
    if line:
        passport += " "+line
    else:
        if all([x in passport for x in required_fields]):
            valid += 1
        passport = ""
if all([x in passport for x in required_fields]):
            valid += 1

print(valid)

216


In [12]:
@dataclass
class Passport:
    byr: int
    iyr: int
    eyr: int
    hgt: str
    hcl: str
    ecl: Literal["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
    pid: str
    cid: Optional[str] = None

    @classmethod
    def from_str(cls, input):
        fields = {k: v for k, v in [field.split(":") for field in input.split()]}

        return cls(**fields)
    
    def is_valid(self) -> bool:

        try:
            assert 1920 <= self.byr and self.byr <= 2002

            assert 2010 <= self.iyr and self.iyr <= 2020

            assert 2020 <= self.eyr and self.eyr <= 2030

            if self.hgt[:2] == "cm":
                height = int(self.hgt[:-2])
                assert 150 <= height and height <= 193
            elif self.hgt[:2] == "in":
                height = int(self.hgt[:-2])
                assert 59 <= height and height <= 76

            assert re.match("#[0-9a-f]{6}", self.hcl)

            assert re.match("[0-9]{9}", self.pid)


        except AssertionError:
            return False
        
        return True

In [13]:
valid = 0
buffer = ""
for line in lines(4):
    if line:
        buffer += " "+line
    else:
        try:
            passport = Passport.from_str(buffer)
            valid += passport.is_valid()
        except:
            pass

try:
    passport = Passport.from_str(buffer)
    valid += passport.is_valid()
except:
    pass

print(valid)

0


In [14]:
passport = Passport.from_str("iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719")
passport.is_valid()

TypeError: '<=' not supported between instances of 'int' and 'str'